In [1]:
!pip install hdbscan
!pip install umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.5 MB/s eta 0:00:00


In [6]:
import re
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hdbscan
import umap
import nltk
import time

from matplotlib.colors import ListedColormap
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN, OPTICS
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.preprocessing import StandardScaler
# Escalar embeddings

nltk.download("stopwords")
nltk.download("wordnet")

# Cargar un modelo multilingüe, adecuado para español
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
'''
Pasa todo a minusculas
remueve urls: No util
remueve signos de puntuacion: Para simplicidad de la gestion de palabras
remueve los tags (@username): No util
remueve las stopwords: No aportan mucho valor para nuestros modelos en NLP
lematiza los datos, es decir, los pone en su forma base: Para favorecer el procesamiento de los datos
remueve espacios de mas: Logico, dado que solo generan mas ruido en nuestro texto

Esta funcion es similar a la mostrada en clase, pero reestructura con expresiones
regulares para mayor simplicidad y orden

'''
lemmatizer = WordNetLemmatizer()
def preprocess_string(input_string):
  input_string = input_string.lower()
  input_string = re.sub(r"@\w+", '', input_string)
  input_string = re.sub(r"http\S+|www\S+|https\S+", '', input_string, flags=re.MULTILINE)
  input_string = re.sub(r"[^a-zA-Z0-9\s]", '', input_string)
  input_string = re.sub(r'\s+', ' ', input_string).strip()

  # El otro filtro de remocion de URLs, remocion de stopwords y lematizacion
  split_string = input_string.split(" ")
  split_string = [word for word in split_string if "://" not in word]
  split_string = [word for word in split_string if word not in stopwords.words('spanish')]
  split_string = [lemmatizer.lemmatize(word) for word in split_string]
  output_string = " ".join(split_string)

  return output_string

In [7]:
df = pd.read_csv("2019_info.csv", index_col=0)
# respuestas = list(df.t1txt)
df['clean'] = df.t1txt.apply(preprocess_string)

## Exp 1: Scaler + UMAP + Iterative OPTICS (cosine)

In [ ]:
from sklearn.cluster import OPTICS

# Generar embeddings

embeddings = model.encode(list(df['clean']))

scaler = StandardScaler()
reducer = umap.UMAP(n_components=80)

embeddings_ = scaler.fit_transform(embeddings)
embeddings_ = reducer.fit_transform(embeddings_)
df_clusters = df.copy()
df_clusters['Cluster'] = -1

for i in range (10):

  df_temp = df_clusters[df_clusters.Cluster == -1].copy()
  embeddings = model.encode(list(df_temp['clean']))
  embeddings_ = scaler.transform(embeddings)
  embeddings_ = reducer.transform(embeddings_)

  # modelo
  optics = OPTICS(min_samples=5, metric='cosine')
  clusters = optics.fit_predict(embeddings)

  # Paso 3: Combinar respuestas con clusters en un DataFrame
  df_temp['Cluster'] = clusters

  # Indicadores:
  n_register = len(df_temp) # numero de registros procesados en la iteracion
  n_clusters = len(np.unique(clusters)) - 1 # numero de clusteres
  n_noise = list(clusters).count(-1) # numero de respuestas no procesadas

  # Mostrar la tabla de conteos
  print(f"iteracion {i}: n_registros {n_register}, n_clusters {n_clusters}, n_outliers {n_noise}")

  # Paso 4: Visualización con t-SNE
  # Reducir dimensionalidad para visualización
  # tsne = TSNE(n_components=2, random_state=42)
  # reduced_embeddings = tsne.fit_transform(embeddings)

  # Actualizar la tabla original:
  offset = df_clusters['Cluster'].nunique() - 1
  df_temp.loc[df_temp.Cluster != -1, 'Cluster'] += offset
  df_clusters.update(df_temp)





/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_optics.py:1083: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


iteracion 0: n_registros 5672, n_clusters 171, n_outliers 1377
iteracion 1: n_registros 1377, n_clusters 47, n_outliers 987
iteracion 2: n_registros 987, n_clusters 24, n_outliers 780
iteracion 3: n_registros 780, n_clusters 11, n_outliers 702
iteracion 4: n_registros 702, n_clusters 6, n_outliers 648
iteracion 5: n_registros 648, n_clusters 2, n_outliers 628
iteracion 6: n_registros 628, n_clusters 3, n_outliers 610
iteracion 7: n_registros 610, n_clusters 1, n_outliers 602
iteracion 8: n_registros 602, n_clusters 1, n_outliers 596
iteracion 9: n_registros 596, n_clusters 1, n_outliers 591


In [ ]:
df_clusters.Cluster.value_counts()

,count
Cluster,
1,1879
-1,648
108,182
101,111
42,92
...,...
161,5
167,5
113,5


## Exp 2: Dinamic Scaler + Dinamic UMAP + Iterative OPTICS (cosine)

In [ ]:
from sklearn.cluster import OPTICS

# Generar embeddings

embeddings = model.encode(list(df['clean']))

scaler = StandardScaler()
reducer = umap.UMAP(n_components=80)
# embeddings_ = scaler.fit_transform(embeddings)
# embeddings_ = reducer.fit_transform(embeddings_)
df_clusters = df.copy()
df_clusters['Cluster'] = -1

for i in range (5):

  df_temp = df_clusters[df_clusters.Cluster == -1].copy()
  embeddings = model.encode(list(df_temp['clean']))

  if i % 3 == 0:
    embeddings_ = scaler.fit_transform(embeddings)
    embeddings_ = reducer.fit_transform(embeddings_)
  else:
    embeddings_ = scaler.transform(embeddings)
    embeddings_ = reducer.transform(embeddings_)

  # modelo
  optics = OPTICS(min_samples=5, metric='cosine')
  clusters = optics.fit_predict(embeddings)

  # Paso 3: Combinar respuestas con clusters en un DataFrame
  df_temp['Cluster'] = clusters

  # Indicadores:
  n_register = len(df_temp) # numero de registros procesados en la iteracion
  n_clusters = len(np.unique(clusters)) - 1 # numero de clusteres
  n_noise = list(clusters).count(-1) # numero de respuestas no procesadas

  # Mostrar la tabla de conteos
  print(f"iteracion {i}: n_registros {n_register}, n_clusters {n_clusters}, n_outliers {n_noise}")

  # Paso 4: Visualización con t-SNE
  # Reducir dimensionalidad para visualización
  # tsne = TSNE(n_components=2, random_state=42)
  # reduced_embeddings = tsne.fit_transform(embeddings)

  # Actualizar la tabla original:
  offset = df_clusters['Cluster'].nunique() - 1
  df_temp.loc[df_temp.Cluster != -1, 'Cluster'] += offset
  df_clusters.update(df_temp)

df_clusters_exp2 = df_clusters.copy()



/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_optics.py:1083: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


iteracion 0: n_registros 5672, n_clusters 171, n_outliers 1377
iteracion 1: n_registros 1377, n_clusters 47, n_outliers 987
iteracion 2: n_registros 987, n_clusters 24, n_outliers 780
iteracion 3: n_registros 780, n_clusters 11, n_outliers 702
iteracion 4: n_registros 702, n_clusters 6, n_outliers 648


## Exp 3: Scaler + UMAP + Iterative OPTICS (euclidean)

In [ ]:
from sklearn.cluster import OPTICS

# Generar embeddings

embeddings = model.encode(list(df['clean']))

scaler = StandardScaler()
embeddings_ = scaler.fit_transform(embeddings)
reducer = umap.UMAP(n_components=80)
embeddings_ = reducer.fit_transform(embeddings_)
df_clusters = df.copy()
df_clusters['Cluster'] = -1

for i in range (5):

  df_temp = df_clusters[df_clusters.Cluster == -1].copy()
  embeddings = model.encode(list(df_temp['clean']))
  embeddings_ = scaler.fit_transform(embeddings)
  embeddings_ = reducer.fit_transform(embeddings_)

  # modelo
  optics = OPTICS(min_samples=5, metric='euclidean')
  clusters = optics.fit_predict(embeddings)

  # Paso 3: Combinar respuestas con clusters en un DataFrame
  df_temp['Cluster'] = clusters

  # Indicadores:
  n_register = len(df_temp) # numero de registros procesados en la iteracion
  n_clusters = len(np.unique(clusters)) - 1 # numero de clusteres
  n_noise = list(clusters).count(-1) # numero de respuestas no procesadas

  # Mostrar la tabla de conteos
  print(f"iteracion {i}: n_registros {n_register}, n_clusters {n_clusters}, n_outliers {n_noise}")

  # Paso 4: Visualización con t-SNE
  # Reducir dimensionalidad para visualización
  # tsne = TSNE(n_components=2, random_state=42)
  # reduced_embeddings = tsne.fit_transform(embeddings)

  # Actualizar la tabla original:
  offset = df_clusters['Cluster'].nunique() - 1
  df_temp.loc[df_temp.Cluster != -1, 'Cluster'] += offset
  df_clusters.update(df_temp)

df_clusters_exp3 = df_clusters.copy()



/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_optics.py:1083: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


iteracion 0: n_registros 5672, n_clusters 156, n_outliers 1540
iteracion 1: n_registros 1540, n_clusters 31, n_outliers 1300
iteracion 2: n_registros 1300, n_clusters 10, n_outliers 1204
iteracion 3: n_registros 1204, n_clusters 5, n_outliers 1171
iteracion 4: n_registros 1171, n_clusters 1, n_outliers 1163


In [ ]:
!pip install --upgrade hdbscan

## Exp 4: Scaler + UMAP + Iterative HDBSCAN (euclidean) (relacionado con el exp 4 ant)

In [19]:
# Generar embeddings

scaler = StandardScaler()
embeddings = model.encode(list(df['clean']))
# embeddings = scaler.fit_transform(embeddings)
reducer = umap.UMAP(n_components=80)
# embeddings_ = reducer.fit_transform(embeddings)

df_clusters = df.copy()
df_clusters['Cluster'] = -1

for i in range (5):

  df_temp = df_clusters[df_clusters.Cluster == -1].copy()
  embeddings = model.encode(list(df_temp['clean']))

  if i == 0:
    embeddings_ = scaler.fit_transform(embeddings)
    embeddings_ = reducer.fit_transform(embeddings_)
  else:
    embeddings_ = scaler.transform(embeddings)
    embeddings_ = reducer.transform(embeddings_)

  # modelo
  start_time = time.time()
  clusterer = hdbscan.HDBSCAN(
      min_cluster_size=10,  # Tamaño mínimo de un cluster
      min_samples=5,       # Influencia de puntos atípicos
      metric='euclidean'   # Métrica de distancia
  )
  clusters = clusterer.fit_predict(embeddings_)
  end_time = time.time()

  # Paso 3: Combinar respuestas con clusters en un DataFrame
  df_temp['Cluster'] = clusters

  # Indicadores:
  n_register = len(df_temp) # numero de registros procesados en la iteracion
  n_clusters = len(np.unique(clusters)) - 1 # numero de clusteres
  n_noise = list(clusters).count(-1) # numero de respuestas no procesadas

  # Mostrar la tabla de conteos
  print(f"iteracion {i}: n_registros {n_register}, n_clusters {n_clusters}, n_outliers {n_noise}")

  df_count = df_temp.copy()
  df_count['Cluster'] = clusters

  # Contar el número de respuestas en cada cluster
  cluster_counts = df_count['Cluster'].value_counts().reset_index()
  cluster_counts.columns = ['Cluster', 'Cantidad']

  # Mostrar la tabla de conteos
  print(cluster_counts)


  if n_clusters == 0:
    break
  print("*** De la iteracion ***")
  filtered_embeddings = [embedding for i, embedding in enumerate(embeddings_) if clusters[i] != -1]
  filtered_clusters = [cluster for cluster in clusters if cluster != -1]
  sil_score = silhouette_score(filtered_embeddings, filtered_clusters, metric='cosine')
  db_index = davies_bouldin_score(filtered_embeddings, filtered_clusters)
  ch_index = calinski_harabasz_score(filtered_embeddings, filtered_clusters)
  execution_time = end_time - start_time
  print(f"Tiempo de ejecución: {execution_time:.4f} segundos")
  print(f"silhouette_score: {sil_score:.4f}")
  print(f"davies_bouldin_score: {db_index:.4f}")
  print(f"calinski_harabasz_score: {ch_index:.4f}")
  # Paso 4: Visualización con t-SNE
  # Reducir dimensionalidad para visualización
  # tsne = TSNE(n_components=2, random_state=42)
  # reduced_embeddings = tsne.fit_transform(embeddings)

  # Actualizar la tabla original:
  offset = df_clusters['Cluster'].nunique() - 1
  df_temp.loc[df_temp.Cluster != -1, 'Cluster'] += offset
  df_clusters.update(df_temp)


  print("*** Global ***")
  df_temp____ = df_clusters[df_clusters.Cluster != -1].copy()
  embeddings____ = model.encode(list(df_temp____['clean']))

  embeddings____ = scaler.transform(embeddings____)
  embeddings____ = reducer.transform(embeddings____)

  sil_score = silhouette_score(embeddings____, df_temp____.Cluster, metric='cosine')
  db_index = davies_bouldin_score(embeddings____, df_temp____.Cluster)
  ch_index = calinski_harabasz_score(embeddings____, df_temp____.Cluster)
  execution_time = end_time - start_time
  print(f"Tiempo de ejecución: {execution_time:.4f} segundos")
  print(f"silhouette_score: {sil_score:.4f}")
  print(f"davies_bouldin_score: {db_index:.4f}")
  print(f"calinski_harabasz_score: {ch_index:.4f}")


df_clusters_it_exp4 = df_clusters.copy()
df_clusters_it_exp4.to_csv('df_clusters_it_exp4.csv', index=True)



iteracion 0: n_registros 5672, n_clusters 114, n_outliers 386
     Cluster  Cantidad
0          3      1866
1         -1       386
2          1       185
3          5       140
4         82       110
..       ...       ...
110       50        10
111       83        10
112       68        10
113      107        10
114       94        10

[115 rows x 2 columns]
*** De la iteracion ***
Tiempo de ejecución: 2.2093 segundos
silhouette_score: 0.9387
davies_bouldin_score: 0.2826
calinski_harabasz_score: 28326.0191
*** Global ***
Tiempo de ejecución: 2.2093 segundos
silhouette_score: 0.3785
davies_bouldin_score: 3.6287
calinski_harabasz_score: 518.0163
iteracion 1: n_registros 386, n_clusters 4, n_outliers 49
   Cluster  Cantidad
0        2       255
1       -1        49
2        0        42
3        1        29
4        3        11
*** De la iteracion ***
Tiempo de ejecución: 0.0266 segundos
silhouette_score: 0.9535
davies_bouldin_score: 0.1638
calinski_harabasz_score: 3140.2496
*** Global **

In [ ]:
df_clusters.Cluster.value_counts()

,count
Cluster,
72,1863
96,790
-1,310
74,182
94,164
...,...
36,10
32,10
92,10


In [ ]:
df_clusters[df_clusters.Cluster == 36]

,t1txt,clean,Cluster
1058,LA CONTAMINACION DE LA MINERIA,contaminacion mineria,36
1081,LA CONTAMINACION DE LA MINERIA,contaminacion mineria,36
1082,LA CONTAMINACION DE LA MINERIA,contaminacion mineria,36
1149,CONTAMINACION DE LA MINERIA,contaminacion mineria,36
1152,CONTAMINACION DE LA MINERIA,contaminacion mineria,36
1645,CONTAMINACION DE LA MINERIA,contaminacion mineria,36
5086,CONTAMINACION DE LA MINERIA,contaminacion mineria,36
5184,PROBLEMAS DE CONTAMINACION POR MINERIA,problemas contaminacion mineria,36
8222,CONTAMINACION SE LA MINERIA,contaminacion mineria,36
8292,LA CONTAMINACION DE LA MINERIA,contaminacion mineria,36


## Exp 5: Scaler + UMAP (cosine) + Iterative HDBSCAN (euclidean)

In [ ]:
from sklearn.cluster import OPTICS

# Generar embeddings

embeddings = model.encode(list(df['clean']))

scaler = StandardScaler()
reducer = umap.UMAP(n_components=80, metric='cosine')
embeddings_ = scaler.fit_transform(embeddings)
embeddings_ = reducer.fit_transform(embeddings_)
df_clusters = df.copy()
df_clusters['Cluster'] = -1

for i in range (15):

  df_temp = df_clusters[df_clusters.Cluster == -1].copy()
  embeddings = model.encode(list(df_temp['clean']))

  embeddings_ = scaler.transform(embeddings)
  embeddings_ = reducer.transform(embeddings_)

  # modelo
  clusterer = hdbscan.HDBSCAN(
      min_cluster_size=10,  # Tamaño mínimo de un cluster
      min_samples=5,       # Influencia de puntos atípicos
      metric='euclidean'   # Métrica de distancia
  )
  clusters = clusterer.fit_predict(embeddings)

  # Paso 3: Combinar respuestas con clusters en un DataFrame
  df_temp['Cluster'] = clusters

  # Indicadores:
  n_register = len(df_temp) # numero de registros procesados en la iteracion
  n_clusters = len(np.unique(clusters)) - 1 # numero de clusteres
  n_noise = list(clusters).count(-1) # numero de respuestas no procesadas

  # Mostrar la tabla de conteos
  print(f"iteracion {i}: n_registros {n_register}, n_clusters {n_clusters}, n_outliers {n_noise}")

  # Paso 4: Visualización con t-SNE
  # Reducir dimensionalidad para visualización
  # tsne = TSNE(n_components=2, random_state=42)
  # reduced_embeddings = tsne.fit_transform(embeddings)

  # Actualizar la tabla original:
  offset = df_clusters['Cluster'].nunique() - 1
  df_temp.loc[df_temp.Cluster != -1, 'Cluster'] += offset
  df_clusters.update(df_temp)

df_clusters_exp5 = df_clusters.copy()



iteracion 0: n_registros 5672, n_clusters 83, n_outliers 1621
iteracion 1: n_registros 1621, n_clusters 13, n_outliers 1214
iteracion 2: n_registros 1214, n_clusters 2, n_outliers 405
iteracion 3: n_registros 405, n_clusters 2, n_outliers 338
iteracion 4: n_registros 338, n_clusters 2, n_outliers 310
iteracion 5: n_registros 310, n_clusters 0, n_outliers 310
iteracion 6: n_registros 310, n_clusters 0, n_outliers 310
iteracion 7: n_registros 310, n_clusters 0, n_outliers 310
iteracion 8: n_registros 310, n_clusters 0, n_outliers 310
iteracion 9: n_registros 310, n_clusters 0, n_outliers 310
iteracion 10: n_registros 310, n_clusters 0, n_outliers 310
iteracion 11: n_registros 310, n_clusters 0, n_outliers 310
iteracion 12: n_registros 310, n_clusters 0, n_outliers 310
iteracion 13: n_registros 310, n_clusters 0, n_outliers 310
iteracion 14: n_registros 310, n_clusters 0, n_outliers 310


## Exp 6: Scaler + UMAP (cosine) + Iterative OPTICS (cosine)

In [ ]:
from sklearn.cluster import OPTICS

# Generar embeddings

embeddings = model.encode(list(df['clean']))

scaler = StandardScaler()
reducer = umap.UMAP(n_components=80, metric='cosine')

embeddings_ = scaler.fit_transform(embeddings)
embeddings_ = reducer.fit_transform(embeddings_)
df_clusters = df.copy()
df_clusters['Cluster'] = -1

for i in range (5):

  df_temp = df_clusters[df_clusters.Cluster == -1].copy()
  embeddings = model.encode(list(df_temp['clean']))
  embeddings_ = scaler.transform(embeddings)
  embeddings_ = reducer.transform(embeddings_)

  # modelo
  optics = OPTICS(min_samples=5, metric='cosine')
  clusters = optics.fit_predict(embeddings)

  # Paso 3: Combinar respuestas con clusters en un DataFrame
  df_temp['Cluster'] = clusters

  # Indicadores:
  n_register = len(df_temp) # numero de registros procesados en la iteracion
  n_clusters = len(np.unique(clusters)) - 1 # numero de clusteres
  n_noise = list(clusters).count(-1) # numero de respuestas no procesadas

  # Mostrar la tabla de conteos
  print(f"iteracion {i}: n_registros {n_register}, n_clusters {n_clusters}, n_outliers {n_noise}")

  # Paso 4: Visualización con t-SNE
  # Reducir dimensionalidad para visualización
  # tsne = TSNE(n_components=2, random_state=42)
  # reduced_embeddings = tsne.fit_transform(embeddings)

  # Actualizar la tabla original:
  offset = df_clusters['Cluster'].nunique() - 1
  df_temp.loc[df_temp.Cluster != -1, 'Cluster'] += offset
  df_clusters.update(df_temp)


df_clusters_exp6 = df_clusters.copy()


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_optics.py:1083: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


iteracion 0: n_registros 5672, n_clusters 173, n_outliers 1353
iteracion 1: n_registros 1353, n_clusters 44, n_outliers 982
iteracion 2: n_registros 982, n_clusters 23, n_outliers 810
iteracion 3: n_registros 810, n_clusters 16, n_outliers 685
iteracion 4: n_registros 685, n_clusters 6, n_outliers 648


In [ ]:
df_clusters_exp6.Cluster.value_counts().head(20)

,count
Cluster,
1,1855
-1,648
111,182
104,111
41,89
148,82
118,58
99,49
2,46


# Experimento con LLM como agente de decisión

In [ ]:
from huggingface_hub import login

# Introduce tu token personal
hugging_face_token = ""

# Loguearte en Hugging Face
login(token=hugging_face_token)
print("Logged in successfully!")

Logged in successfully!


In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, GenerationConfig

class Llama3:
    def __init__(self, model_path):
        self.model_id = model_path

        # Cargar el tokenizador
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_id)

        # Cargar el modelo
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_id,
            torch_dtype=torch.float16,
            device_map="auto",
        )

        # Verificar y asignar tokens especiales
        if self.tokenizer.eos_token_id is None:
            self.tokenizer.eos_token_id = self.tokenizer.convert_tokens_to_ids("</s>")
        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

        # Crear configuración de generación
        self.generation_config = GenerationConfig.from_pretrained(self.model_id)
        self.generation_config.eos_token_id = self.tokenizer.eos_token_id
        self.generation_config.pad_token_id = self.tokenizer.pad_token_id

        # Crear pipeline
        self.pipeline = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            config=self.generation_config
        )

    def get_response(self, query, message_history=[], max_tokens=100000, temperature=0.7, top_p=0.9):
        user_prompt = message_history + [{"role": "user", "content": query}]
        prompt = self.tokenizer.apply_chat_template(
            user_prompt, tokenize=False, add_generation_prompt=True
        )

        # Generar texto
        outputs = self.pipeline(
            prompt,
            max_new_tokens=max_tokens,
            eos_token_id=self.tokenizer.eos_token_id,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
        )

        response = outputs[0]["generated_text"][len(prompt):]
        return response, user_prompt + [{"role": "assistant", "content": response}]

    def chatbot(self, system_instructions=""):
        conversation = [{"role": "system", "content": system_instructions}]
        while True:
            user_input = input("User: ")
            if user_input.lower() in ["exit", "quit"]:
                print("Cerrando el chatbot. Goodbye!")
                break
            response, conversation = self.get_response(user_input, conversation)
            print(f"Assistant: {response}")

    def chatbot_without_memory(self, system_instructions=""):
        conversation = [{"role": "system", "content": system_instructions}]
        while True:
            user_input = input("User: ")
            if user_input.lower() in ["exit", "quit"]:
                print("Cerrando el chatbot. Goodbye!")
                break
            response, _ = self.get_response(user_input, conversation)
            print(f"Assistant: {response}")

    def prompt_without_memory(self, prompt, system_instructions=""):
        conversation = [{"role": "system", "content": system_instructions}]
        response, _ = self.get_response(prompt, conversation)
        return response

In [ ]:
# if _name_ == "_main_":
torch.cuda.empty_cache()

bot = Llama3("meta-llama/Meta-Llama-3-8B-Instruct")
# bot.chatbot_without_memory()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
INSTRUCTION = """
INPUT:
Eres un modelo que recibiria informacion con la estructura
indice1 | texto1
indice2 | texto2
indice3 | texto3
...
indiceN | textoN

PROCESO QUE DEBES REALIZAR:
Solo debes evaluar las siguientes cosas:
 - similarity: Si son similares o no (Valores: "True" cuando es similar o "False" si no lo es)
 - cluster_name: El nombre que deberia tener todo este conjunto de textos

RESPUESTA:

Devolver lo procesado segun el siguiente formato. No necesitas explicar nada.
similarity|cluster_name

ejm: True|La corrupcion en el Perú
"""

In [ ]:
result = ''
for index, row in df_clusters_exp6[df_clusters_exp6.Cluster == 24].iterrows():
  result += f"{index} | {row['clean']}\n"

print(result)

398 | falta pavimentacion carreteras
3325 | falta pavimentacion mejoramiento carretera
3826 | falta pavimentacion calles
4048 | falta pavimentacion caminos caserio
4734 | falta pavimentacion calles



In [ ]:
bot.prompt_without_memory(result, INSTRUCTION)

KeyboardInterrupt: 

In [ ]:
df_clusters_exp6[df_clusters_exp6.Cluster == 149]

In [ ]:
df_clusters_exp6.to_csv('df_clusters_exp6.csv', index=True)